# December, 19th 2020

**Motivation**: Simple dimensionality analysis using PCA.<br>
**Result**: Data is high dimensional.  We need ??? dimensions to explain 70% of variance for all responses from all trial types. We need ??? dims to explain 70% variance of hit trials and so on

In [ ]:
# HIDE CODE

import os
import sys
import h5py
import pickle
import numpy as np
import pandas as pd
from os.path import join as pjoin
from copy import deepcopy as dc
from tqdm.notebook import tqdm
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')

github_path = pjoin(os.environ['HOME'], 'Dropbox/git/A1ToneDiscrimination/')
sys.path.insert(0, github_path)
from utils.process import *
from utils.generic_utils import *
from utils.plot_functions import *